In [ ]:
import numpy as np
import librosa
import os
from matplotlib import pyplot as plt
!pip install pretty_midi
import pretty_midi 
pretty_midi.pretty_midi.MAX_TICK = 1e10

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#preparing the data from the original audio files
entries = os.scandir('/content/drive/MyDrive/train_source')#loading the data to process which we are going to use for training 

for file in entries:
    file_name,file_extensions=os.path.splitext(file)

    if file_extensions=='.wav':
        sr=16000
        y, sr = librosa.load(file, sr=sr)
        hop_length=512
        n_fft=2048
        MFCC=librosa.feature.mfcc(y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mfcc=13)#using the librosa library to calculate the mel spectrogram of the audio file which is returned as a 2D vector 
        c=len(y) # it is being stored because it will be used for the frequency while making the piano roll file
        MFCC=np.abs(MFCC)# the returned 2d vector is a complex quantity 
        MFCC=librosa.amplitude_to_db(MFCC)# taking the amplitude on logarithmic scale gives a better visualization of data
        b=np.shape(MFCC)[1]
        print(np.shape(np.transpose(MFCC)))
        entries1 = os.scandir('/content/drive/MyDrive/train_source')#it is being loaded again so that we can get the corresponding midi file to the wav file which we have just used
        for file1 in entries1:
          file_name1,file_extensions1=os.path.splitext(file1)

          if file_name1==file_name and file_extensions1==".mid":
            file2=file_name + file_extensions1
            pm = pretty_midi.PrettyMIDI(file2)
            a=pm.instruments[0].get_piano_roll(float(sr*float(b)/float(c)))# The Sampling frequency has been taken such that its shape and the mel files shape is same 
            a = (a[21:109, :b] > 0)#we only need the frequency of the piano and converting non binary data to 0 and 1
            a=np.transpose(a)
            print(np.shape(a))
            out = '/content/drive/MyDrive/trainmel/melmid/mid_'+str(file_name1[40:])+'mid'+'.npy'
            np.save(out,a)
            print('wrote file', out)
            out = '/content/drive/MyDrive/trainmel/melwav/wav_'+str(file_name[40:])+'.npy'
            MFCC=np.transpose(MFCC)
            if a.shape[0]<MFCC.shape[0]:#cutting out little extra part in MFCC arrays
              MFCC=MFCC[:a.shape[0],:]
            print(np.shape(MFCC))  
            np.save(out,MFCC)
            print("saving", out)

  

(7468, 13)
(7405, 88)
wrote file /content/drive/MyDrive/trainmel/melmid/mid__MUS-alb_se2_StbgTGd2mid.npy
(7405, 13)
saving /content/drive/MyDrive/trainmel/melwav/wav__MUS-alb_se2_StbgTGd2.npy
(12058, 13)
(11995, 88)
wrote file /content/drive/MyDrive/trainmel/melmid/mid__MUS-bk_xmas1_StbgTGd2mid.npy
(11995, 13)
saving /content/drive/MyDrive/trainmel/melwav/wav__MUS-bk_xmas1_StbgTGd2.npy
(7215, 13)
(7152, 88)
wrote file /content/drive/MyDrive/trainmel/melmid/mid__MUS-bk_xmas4_StbgTGd2mid.npy
(7152, 13)
saving /content/drive/MyDrive/trainmel/melwav/wav__MUS-bk_xmas4_StbgTGd2.npy
(12370, 13)
(12307, 88)
wrote file /content/drive/MyDrive/trainmel/melmid/mid__MUS-bk_xmas5_StbgTGd2mid.npy
(12307, 13)
saving /content/drive/MyDrive/trainmel/melwav/wav__MUS-bk_xmas5_StbgTGd2.npy
(3595, 13)
(3532, 88)
wrote file /content/drive/MyDrive/trainmel/melmid/mid__MUS-bor_ps6_StbgTGd2mid.npy
(3532, 13)
saving /content/drive/MyDrive/trainmel/melwav/wav__MUS-bor_ps6_StbgTGd2.npy
(3531, 13)
(3468, 88)
wrote 

In [ ]:
path1='/content/drive/MyDrive/trainmel/melwav'

def concatenate_wav(path):#we are dividing the number of features of the mel file into batches of 100 and stacking them up to create a 3D array 
  entries = os.scandir(path)
  d=np.zeros((1,100,13))#initialising the array with the correct shape 

  for file in entries:
      a=np.load(file)
      a=a[np.newaxis,:,:]# adding a new axis to the original files
      for b in range(int(np.shape(a)[1]/100)):
          
          c=a[:,b*100:(b*100)+100,:]# dividing them into batches of 100
          d=np.concatenate((d,c), axis=0)# stacking them up on the first axis 
  out = '/content/drive/MyDrive/trainmel/melwav/concatenate'+'wav'+'.npy'
  d=d[1:,:,:]# array was initialise with zeros so excluding that part 

concatenate_wav(path1)

path='/content/drive/MyDrive/trainmel/melmid'

def concatenate_mid(path):#we are dividing the number of features of the pianoroll file into batches of 100 and stacking them up to create a 3D array 
  entries = os.scandir(path)
  d=np.zeros((1,100,88))
  l=0
  for file in entries:
      a=np.load(file)
      a=a[np.newaxis,:,:]
      for b in range(int(np.shape(a)[1]/100)):
            
          c=a[:,b*100:(b*100)+100,:]
          d=np.concatenate((d,c), axis=0)   

          
  out = '/content/drive/MyDrive/trainmel/melmid/concatenate'+'mid'+'.npy'
  d=d[1:,:,:]
  np.save(out,d)  

concatenate_mid(path)


In [ ]:
#preparing the data from the original audio files
entries = os.scandir('/content/drive/MyDrive/test_source')#loading the data to process which we are going to use for training 

for file in entries:
    file_name,file_extensions=os.path.splitext(file)

    if file_extensions=='.wav':
        sr=16000
        y, sr = librosa.load(file, sr=sr)
        hop_length=512
        n_fft=2048
        MFCC=librosa.feature.mfcc(y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mfcc=13)#using the librosa library to calculate the mel of the audio file which is returned as a 2D vector 
        c=len(y) # it is being stored because it will be used for the frequency while making the piano roll file
        MFCC=np.abs(MFCC)# the returned 2d vector is a complex quantity 
        MFCC=librosa.amplitude_to_db(MFCC)# taking the amplitude on logarithmic scale gives a better visualization of data
        b=np.shape(MFCC)[1]

        entries1 = os.scandir('/content/drive/MyDrive/test_source')#it is being loaded again so that we can get the corresponding midi file to the wav file which we have just used
        for file1 in entries1:
          file_name1,file_extensions1=os.path.splitext(file1)

          if file_name1==file_name and file_extensions1==".mid":
            file2=file_name + file_extensions1
            pm = pretty_midi.PrettyMIDI(file2)
            a=pm.instruments[0].get_piano_roll(float(sr*float(b)/float(c)))# The Sampling frequency has been taken such that its shape and the mel files shape is same 
            a = (a[21:109, :b] > 0)#we only need the frequency of the piano and converting non binary data to 0 and 1
            a=np.transpose(a)
            out = '/content/drive/MyDrive/testmel/melmid/mid_'+str(file_name1[40:])+'mid'+'.npy'
            np.save(out,a)
            out = '/content/drive/MyDrive/testmel/melwav/wav_'+str(file_name[40:])+'.npy'
            MFCC=np.transpose(MFCC)
            if a.shape[0]<MFCC.shape[0]:#cutting out little extra part in MFCC arrays
              MFCC=MFCC[:a.shape[0],:]
            np.save(out,MFCC)

  

In [ ]:
path1='/content/drive/MyDrive/testmel/melwav'

def concatenate_wav(path):#we are dividing the number of features of the mel file into batches of 100 and stacking them up to create a 3D array 
  entries = os.scandir(path)
  d=np.zeros((1,100,13))#initialising the array with the correct shape 

  for file in entries:
      a=np.load(file)
      a=a[np.newaxis,:,:]# adding a new axis to the original files
      for b in range(int(np.shape(a)[1]/100)):
          
          c=a[:,b*100:(b*100)+100,:]# dividing them into batches of 100
          d=np.concatenate((d,c), axis=0)# stacking them up on the first axis 
  out = '/content/drive/MyDrive/testmel/melwav/concatenate'+'wav'+'.npy'
  d=d[1:,:,:]# array was initialise with zeros so excluding that part 

  np.save(out,d)  

concatenate_wav(path1)

path='/content/drive/MyDrive/testmel/melmid'

def concatenate_mid(path):#we are dividing the number of features of the pianoroll file into batches of 100 and stacking them up to create a 3D array 
  entries = os.scandir(path)
  d=np.zeros((1,100,88))
  l=0
  for file in entries:
      a=np.load(file)
      a=a[np.newaxis,:,:]
      for b in range(int(np.shape(a)[1]/100)):
            
          c=a[:,b*100:(b*100)+100,:]
          d=np.concatenate((d,c), axis=0)
         
  out = '/content/drive/MyDrive/testmel/melmid/concatenate'+'mid'+'.npy'
  d=d[1:,:,:]
  np.save(out,d)  

concatenate_mid(path)
